# Imports and get project

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm, colors
import glob

from numba import njit, vectorize, float32

from typing import Callable, Optional, Union

import hoomd
import gsd.hoomd
import freud

import sys
import time
import pickle
import gc
import warnings
import copy
import pathlib

import os
import sys

import signac

from dataclasses import dataclass
from dataclasses_json import dataclass_json

In [3]:
from monk import nb, prep, pair, render, utils, workflow, grid

config = workflow.get_config()

In [4]:
project: signac.Project = signac.init_project(name="local-yield", root=config['root'])

# Initialize statepoints

In [17]:
project.doc["avail_seed"] = 0

In [18]:
N = 2**15  # 32758
phis = [1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4]

i = 0
for replica in range(10):
    for phi in phis:
        sp = {"N": N, "replica": replica, "phi": phi}
        job = project.open_job(sp)
        if "init" not in job.doc:
            job.init()
            job.doc["init"] = True
            job.doc["seed"] = project.doc["avail_seed"]
            project.doc["avail_seed"] += 1

In [19]:
for job in project:
    print(job.sp, job.doc)

{'N': 32768, 'replica': 3, 'phi': 1.35} {'init': True, 'seed': 26}
{'N': 32768, 'replica': 5, 'phi': 1.35} {'init': True, 'seed': 40}
{'N': 32768, 'replica': 3, 'phi': 1.15} {'init': True, 'seed': 22}
{'N': 32768, 'replica': 2, 'phi': 1.1} {'init': True, 'seed': 14}
{'N': 32768, 'replica': 1, 'phi': 1.35} {'init': True, 'seed': 12}
{'N': 32768, 'replica': 3, 'phi': 1.4} {'init': True, 'seed': 27}
{'N': 32768, 'replica': 4, 'phi': 1.4} {'init': True, 'seed': 34}
{'N': 32768, 'replica': 6, 'phi': 1.35} {'init': True, 'seed': 47}
{'N': 32768, 'replica': 2, 'phi': 1.15} {'init': True, 'seed': 15}
{'N': 32768, 'replica': 6, 'phi': 1.25} {'init': True, 'seed': 45}
{'N': 32768, 'replica': 5, 'phi': 1.15} {'init': True, 'seed': 36}
{'N': 32768, 'replica': 5, 'phi': 1.2} {'init': True, 'seed': 37}
{'N': 32768, 'replica': 9, 'phi': 1.2} {'init': True, 'seed': 65}
{'N': 32768, 'replica': 1, 'phi': 1.25} {'init': True, 'seed': 10}
{'N': 32768, 'replica': 7, 'phi': 1.35} {'init': True, 'seed': 54}


# Generate initial simulation states and quench to inherent structure

In [20]:
# for job in project:
#     if "seed" not in job.doc:
#         job.doc["seed"] = project.doc["avail_seed"]
#         project.doc["avail_seed"] += 1

In [21]:
for idx, job in enumerate(project):
    print(idx, job.sp)
    sp = job.sp
    doc = job.doc

    # if "quenched" in doc:
    #     continue

    sim = prep.quick_sim(sp["N"], sp["phi"], hoomd.device.GPU(), dim=2, ratios=[50, 50], diams=[14/12, 10/12], seed=doc["seed"])

    integrator = hoomd.md.Integrator(1e-2)
    tree = hoomd.md.nlist.Tree(0.3)
    hertz = pair.bi_hertz(tree)
    nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=10.0, tau=1.0)
    integrator.forces = [hertz]
    integrator.methods = [nvt]
    sim.operations.integrator = integrator

    sim.run(10_000)

    sim.operations.integrator = None
    del integrator, hertz, nvt, tree

    hoomd.write.GSD.write(sim.state, job.fn("init.gsd"))

    fire = hoomd.md.minimize.FIRE(1e-2, 1e-3, 1.0, 1e-3)
    tree = hoomd.md.nlist.Tree(0.3)
    hertz = pair.bi_hertz(tree)
    nve = hoomd.md.methods.NVE(filter=hoomd.filter.All())
    fire.forces = [hertz]
    fire.methods = [nve]
    sim.operations.integrator = fire
    while not fire.converged:
        sim.run(10_000)

    hoomd.write.GSD.write(sim.state, job.fn("quenched.gsd"))
    doc["quenched"] = True

0 {'N': 32768, 'replica': 3, 'phi': 1.35}
1 {'N': 32768, 'replica': 5, 'phi': 1.35}
2 {'N': 32768, 'replica': 3, 'phi': 1.15}
3 {'N': 32768, 'replica': 2, 'phi': 1.1}
4 {'N': 32768, 'replica': 1, 'phi': 1.35}
5 {'N': 32768, 'replica': 3, 'phi': 1.4}
6 {'N': 32768, 'replica': 4, 'phi': 1.4}
7 {'N': 32768, 'replica': 6, 'phi': 1.35}
8 {'N': 32768, 'replica': 2, 'phi': 1.15}
9 {'N': 32768, 'replica': 6, 'phi': 1.25}
10 {'N': 32768, 'replica': 5, 'phi': 1.15}
11 {'N': 32768, 'replica': 5, 'phi': 1.2}
12 {'N': 32768, 'replica': 9, 'phi': 1.2}
13 {'N': 32768, 'replica': 1, 'phi': 1.25}
14 {'N': 32768, 'replica': 7, 'phi': 1.35}
15 {'N': 32768, 'replica': 2, 'phi': 1.25}
16 {'N': 32768, 'replica': 8, 'phi': 1.3}
17 {'N': 32768, 'replica': 4, 'phi': 1.1}
18 {'N': 32768, 'replica': 9, 'phi': 1.1}
19 {'N': 32768, 'replica': 7, 'phi': 1.3}
20 {'N': 32768, 'replica': 0, 'phi': 1.4}
21 {'N': 32768, 'replica': 9, 'phi': 1.35}
22 {'N': 32768, 'replica': 3, 'phi': 1.25}
23 {'N': 32768, 'replica': 3, '

In [5]:
for idx, job in enumerate(project):
    
    sp = job.sp
    doc = job.doc

    if "quenched-more" in doc:
        continue
    print(idx, job.sp)

    sim = hoomd.Simulation(hoomd.device.GPU(), seed=doc["seed"])
    sim.create_state_from_gsd(job.fn("quenched.gsd"))

    fire = hoomd.md.minimize.FIRE(1e-2, 1e-4, 1.0, 1e-4)
    tree = hoomd.md.nlist.Tree(0.3)
    hertz = pair.bi_hertz(tree)
    nve = hoomd.md.methods.NVE(filter=hoomd.filter.All())
    fire.forces = [hertz]
    fire.methods = [nve]
    sim.operations.integrator = fire
    jdx = 0
    while not fire.converged:
        print(jdx)
        sim.run(10_000)
        jdx += 1

    hoomd.write.GSD.write(sim.state, job.fn("quenched-more.gsd"))
    doc["quenched-more"] = True
print("done")

done
